In [ ]:
! wget https://raw.githubusercontent.com/PAE-ITBA-ML-2019/Regularizacion/master/reg_helper.py
! wget https://raw.githubusercontent.com/PAE-ITBA-ML-2019/Regularizacion/master/fnn_helper.py
! wget https://raw.githubusercontent.com/PAE-ITBA-ML-2019/Regularizacion/master/draw_nn.py
! wget https://raw.githubusercontent.com/PAE-ITBA-ML-2019/Regularizacion/master/regularization_helper.py
 
! wget https://github.com/PAE-ITBA-ML-2019/Regularizacion/raw/master/data.zip
! unzip data.zip


# LASSO 
**(Least Absolute Shrinkage and Selection Operator) -L1 regularization-**
- L2 primero reduce los pesos con magnitudes altas por que para mismo delta de reducción, si la magnitud es alta, la reducción es mayor. L1 en cambio por ser lineal da igual si la magnitud es alta o baja.
- L2 no da "sparsity" va bajando todos al mismo tiempo.
- Selector de parametros (Feature selection) - Puede tratar los pesos en forma independiente. Cuando un peso es cero, No importa la relación con el resto como en L2. Sí depende de la función de costo
- $g(w) = \|w\|$ 
- Sparse solution

<img src="images/lasso.png" alt="Drawing" style="width:60%;"/>

# LASSO vs RIDGE

In [ ]:
from sklearn.linear_model import Lasso, Ridge
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from reg_helper import get_lin_reg_pol, get_ridge_weights, get_MLE_MAP_weights, plt_lin_reg_gauss, get_curves

In [ ]:
df = pd.read_csv('./data/alturas-pesos-mils-train.csv')
data = df[['Altura', 'Peso']].values

In [ ]:
plt.scatter(data[:,0], data[:,1], s=1)
plt.show()

# Ejemplo con polinomio de orden 7 tomando de a 20 observaciones

In [ ]:
N = 20 # Cantidad de observaciones
i_sel = 3 # Tomadas de a N cual de las 8000/N tomo
order = 7 # Orden del polinomio
ptos=100 # Resolución de recta
al = np.linspace(data[:,0].min(),data[:,0].max(), ptos)
alturas_pol, mean, std = get_lin_reg_pol(data[:,0], order, normalize=True)

### Alpha es lambda (Notación Sklearn)

In [ ]:
alphas = np.logspace(-3, 3, 100)

In [ ]:
w_MSE = get_ridge_weights(alturas_pol[i_sel*N:(i_sel+1)*N, :], data[:,1][i_sel*N:(i_sel+1)*N], lamb = 0).reshape(1,-1)

In [ ]:
w_MSE.T.shape

In [ ]:
ws_RIDGE = np.zeros([len(alphas), order+1])
ws_LASSO = np.zeros([len(alphas), order+1])
for i, alpha in enumerate(alphas):
    clf_ridge = Ridge(alpha = alpha, fit_intercept=False)
    clf_ridge.fit(alturas_pol[i_sel*N:(i_sel+1)*N, :], data[:,1][i_sel*N:(i_sel+1)*N])
    w_RIDGE = np.array(list(clf_ridge.coef_))
    w_RIDGE = get_ridge_weights(alturas_pol[i_sel*N:(i_sel+1)*N, :], data[:,1][i_sel*N:(i_sel+1)*N], lamb = alpha).reshape(1,-1)
    clf_lasso = Lasso(alpha = alpha, fit_intercept=False, tol=0.0001, max_iter=100000)
    clf_lasso.fit(alturas_pol[i_sel*N:(i_sel+1)*N, :], data[:,1][i_sel*N:(i_sel+1)*N])
    w_LASSO = np.array(list(clf_lasso.coef_))
    ws_RIDGE[i,:] = w_RIDGE
    ws_LASSO[i,:] = w_LASSO

### Aclaración: No estoy graficando el intercept

In [ ]:
plt.figure(figsize=(20,5))
plt.semilogx(alphas, ws_RIDGE[:,1:])
plt.title("Valor de los $w_i$'s en función de $\lambda$ -Ridge-")
plt.show()

plt.figure(figsize=(20,5))
plt.semilogx(alphas[20:], ws_RIDGE[20:,1:])
plt.title("Valor de los $w_i$'s en función de $\lambda$ -Ridge- Zoom")
plt.show()

plt.figure(figsize=(20,5))
plt.semilogx(alphas, ws_LASSO[:,1:])
plt.title("Valor de los $w_i$'s en función de $\lambda$ -LASSO- ")
plt.show()

Notar en la gráfica superior como algunos pesos se van a cero (Sparsity)

In [ ]:
from sklearn.linear_model import LinearRegression
clf = LinearRegression()
clf.fit(data[:,0][i_sel*N:(i_sel+1)*N].reshape(-1,1), data[:,1][i_sel*N:(i_sel+1)*N])
w_LR = np.array([clf.intercept_, clf.coef_[0]]).reshape(2,1)
print(w_LR)
pe_LR = get_lin_reg_pol(al, normalize=False, order=1).dot(w_LR)

# Regresión de ridge en función de lambda

In [ ]:
plt.figure(figsize=(20,10))
al = np.linspace(data[:,0][i_sel*N:(i_sel+1)*N].min(),data[:,0][i_sel*N:(i_sel+1)*N].max(), ptos)
al_line = get_lin_reg_pol(al, normalize=False, order=order, mean=mean, std=std)
paso = 8
plt.plot(al, pe_LR, 'k', label='regresión lineal')
plt.plot(al, al_line.dot(w_MSE.T), 'k', ls='-.',label='$\lambda$=0')
for i, w in enumerate(ws_RIDGE[::paso]):
    pe_RIDGE = al_line.dot(w)
    plt.plot(al, pe_RIDGE, label='$\lambda$='+str(np.round(alphas[i*5]*1000)/1000))

plt.legend()
plt.scatter(data[:,0][i_sel*N:(i_sel+1)*N], data[:,1][i_sel*N:(i_sel+1)*N])
plt.show()

Se ve claramente el aumento de bias con el aumento de lambda

# Regresión L1 en función de lambda

In [ ]:
plt.figure(figsize=(20,10))
al_line = get_lin_reg_pol(al, normalize=False, order=order, mean=mean, std=std)
paso = 8
plt.plot(al, pe_LR, 'k', label='regresión lineal')
plt.plot(al, al_line.dot(w_MSE.T), 'k', ls='-.',label='$\lambda$=0')
for i, w in enumerate(ws_LASSO[::paso]):
    pe_LASSO = al_line.dot(w)
    plt.plot(al, pe_LASSO, label='$\lambda$='+str(np.round(alphas[i*5]*1000)/1000))

plt.legend()
plt.scatter(data[:,0][i_sel*N:(i_sel+1)*N], data[:,1][i_sel*N:(i_sel+1)*N])
plt.show()

# ElasticNet
-  Combinacion de L1 y L2

# 4.4 Sparcity y LO regularization:

$w_{RR} = arg \min\| \hat{y}-y_i \|^2 + \lambda \|w\|^d$

- L0 cuenta la cantidad de $w$'s distinto de cero (d=0)
- Si d esta entre 0 y 1 aumenta la esparcibidad (sparsity) pero la funcion de costo no es convexa